In [1]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import from_bounds

# --- Input ---
soilgrids_vrt = "/vsicurl/https://files.isric.org/soilgrids/latest/data/ocs/ocs_0-30cm_mean.vrt"
output_tif = "angola_ocs_0_30cm.tif"

# Angola bounding box in meters
xmin, ymin = 143882.578, 7995805.043
xmax, ymax = 1495882.578, 9516805.043
dst_width = 1352
dst_height = 1521

# Target CRS (meters)
dst_crs = "EPSG:32733"  # UTM Zone 33s

# Open SoilGrids VRT (in degrees)
with rasterio.open(soilgrids_vrt) as src:
    # Calculate transform for Angola bbox in meters
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    
    # Update metadata
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'width': dst_width,
        'height': dst_height,
        'transform': rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, dst_width, dst_height),
        'driver': 'GTiff'
    })

    # Create output raster
    with rasterio.open(output_tif, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=kwargs['transform'],
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )

print("Download and reproject complete:", output_tif)


Download and reproject complete: angola_ocs_0_30cm.tif
